In [3]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer
from qiskit.circuit import Parameter

# Fix font warnings
plt.rcParams['font.family'] = ['DejaVu Sans', 'Liberation Sans', 'Arial', 'sans-serif']
plt.rcParams['mathtext.default'] = 'regular'

def create_xy_interaction(qc, qubit1, qubit2, beta_param):
    """Add a single XY interaction between two qubits."""
    qc.ry(np.pi/2, qubit1)      # Rotate qubit1 to X-Y plane
    qc.ry(np.pi/2, qubit2)      # Rotate qubit2 to X-Y plane
    qc.cx(qubit1, qubit2)       # First CNOT
    qc.rz(2*beta_param, qubit2) # Z-rotation with 2β parameter
    qc.cx(qubit1, qubit2)       # Second CNOT
    qc.ry(-np.pi/2, qubit1)     # Rotate qubit1 back
    qc.ry(-np.pi/2, qubit2)     # Rotate qubit2 back

def create_6qubit_even_odd_xy_mixer(beta=np.pi/8):
    """Create the 6-qubit even/odd XY mixer with closed ring topology."""
    
    qc = QuantumCircuit(6, name='6Q_EvenOdd_XY_Mixer')
    
    # Create symbolic parameter for display
    beta_param = Parameter('β')
    
    # EVEN LAYER: Connections starting from even indices
    # These form the "backbone" of the ring
    qc.barrier(label='Even Layer')
    
    # (0,1) - First even connection
    create_xy_interaction(qc, 0, 1, beta_param)
    
    # (2,3) - Second even connection  
    create_xy_interaction(qc, 2, 3, beta_param)
    
    # (4,5) - Third even connection
    create_xy_interaction(qc, 4, 5, beta_param)
    
    #qc.barrier()
    
    # ODD LAYER: Connections starting from odd indices
    # These complete the ring connectivity
    qc.barrier(label='Odd Layer')
    
    # (1,2) - First odd connection
    create_xy_interaction(qc, 1, 2, beta_param)
    
    # (3,4) - Second odd connection
    create_xy_interaction(qc, 3, 4, beta_param)
    
    # (5,0) - Third odd connection - CLOSES THE RING!
    create_xy_interaction(qc, 5, 0, beta_param)
    
    # For simulation, we need to bind the parameter to actual values
    try:
        # Try assign_parameters method
        qc_bound = qc.assign_parameters({beta_param: beta})
    except AttributeError:
        # Fallback: create a separate numerical circuit
        qc_bound = QuantumCircuit(6, name='6Q_EvenOdd_XY_Mixer_Bound')
        
        # Even layer
        qc_bound.barrier()
        create_xy_interaction(qc_bound, 0, 1, 2*beta)  # Use numerical value
        create_xy_interaction(qc_bound, 2, 3, 2*beta)
        create_xy_interaction(qc_bound, 4, 5, 2*beta)
        
        qc_bound.barrier()
        
        # Odd layer
        qc_bound.barrier()
        create_xy_interaction(qc_bound, 1, 2, 2*beta)
        create_xy_interaction(qc_bound, 3, 4, 2*beta)
        create_xy_interaction(qc_bound, 5, 0, 2*beta)  # Closes the ring
    
    return qc, qc_bound

def plot_6qubit_even_odd_xy_mixer():
    """Plot the 6-qubit even/odd XY mixer circuit."""
    
    qc_symbolic, qc_bound = create_6qubit_even_odd_xy_mixer()
    
    # Use symbolic circuit for display (shows 2β)
    try:
        # Method 1: Direct matplotlib circuit drawing
        fig, ax = plt.subplots(figsize=(20, 10))
        qc_symbolic.draw(output='mpl', ax=ax, style='default', fold=None, scale=0.8)
        
        plt.tight_layout()
        plt.savefig('6qubit_even_odd_xy_mixer.png', dpi=600, bbox_inches='tight', 
                    facecolor='white', edgecolor='none')
        
        print("✓ Saved circuit as '6qubit_even_odd_xy_mixer.png' (Method 1)")
        plt.show()
        
    except Exception as e:
        print(f"Method 1 failed: {e}")
        
        # Method 2: Use circuit_drawer but handle differently
        try:
            plt.figure(figsize=(20, 10))
            fig = circuit_drawer(qc_symbolic, output='mpl', style='default', fold=None, scale=0.8)
            
            if fig is not None:
                plt.savefig('6qubit_even_odd_xy_mixer.png', dpi=600, bbox_inches='tight', 
                            facecolor='white', edgecolor='none')
                print("✓ Saved circuit as '6qubit_even_odd_xy_mixer.png' (Method 2)")
                plt.show()
            else:
                print("Circuit drawer returned None")
                
        except Exception as e2:
            print(f"Method 2 also failed: {e2}")
            print("Falling back to text-only representation")
    
    # Always show text representation
    print("\nCircuit text representation:")
    print(qc_symbolic.draw(output='text'))
    
    return qc_symbolic, qc_bound

def demonstrate_6qubit_even_odd_effect():
    """Demonstrate what the 6-qubit even/odd XY mixer does."""
    
    try:
        # Try newer Qiskit first
        from qiskit_aer import Aer
        from qiskit import transpile
        use_aer = True
    except ImportError:
        try:
            # Try older Qiskit
            from qiskit import Aer
            from qiskit import transpile
            use_aer = True
        except ImportError:
            # Fall back to basic quantum_info
            from qiskit.quantum_info import Statevector
            use_aer = False
    
    print("\n6-Qubit Even/Odd XY Mixer Effect:")
    print("="*40)
    print("Using β = π/8 for demonstration")
    print("Testing ring connectivity with different initial states")
    print()
    
    # Test states that show the ring connectivity
    test_states = ['000001', '000010', '100000', '010000']
    
    for initial_state in test_states:
        # Create circuit with initial state
        qc = QuantumCircuit(6)
        
        # Prepare initial state
        for i, bit in enumerate(initial_state):
            if bit == '1':
                qc.x(i)
        
        # Add 6-qubit even/odd XY mixer
        _, xy_mixer_bound = create_6qubit_even_odd_xy_mixer(beta=np.pi/8)
        qc = qc.compose(xy_mixer_bound)
        
        if use_aer:
            # Use Aer backend
            backend = Aer.get_backend('statevector_simulator')
            transpiled_qc = transpile(qc, backend)
            job = backend.run(transpiled_qc)
            result = job.result()
            statevector = result.get_statevector()
        else:
            # Use quantum_info Statevector
            statevector = Statevector.from_instruction(qc)
        
        print(f"\nInitial state |{initial_state}⟩:")
        
        # Display final state probabilities (only significant ones)
        probs = np.abs(statevector)**2
        
        # Show top 5 most probable states
        top_indices = np.argsort(probs)[-5:][::-1]
        
        for idx in top_indices:
            if probs[idx] > 0.001:
                binary_state = format(idx, '06b')  # 6-bit binary
                print(f"  |{binary_state}⟩: {probs[idx]:.3f}")

def explain_even_odd_pattern():
    """Explain the even/odd mixer pattern and ring closure."""
    
    print("\nEVEN/ODD MIXER PATTERN:")
    print("="*35)
    
    print("EVEN LAYER (Parallel Pairs):")
    print("  • (0,1): Connect bottom pair")
    print("  • (2,3): Connect middle pair") 
    print("  • (4,5): Connect top pair")
    print("  → Creates 3 separate 2-qubit clusters")
    print()
    
    print("ODD LAYER (Ring Connections):")
    print("  • (1,2): Connect first cluster to second")
    print("  • (3,4): Connect second cluster to third")
    print("  • (5,0): Connect third cluster back to first → CLOSES RING!")
    print("  → Links all clusters into one connected component")
    print()
    
    print("RING TOPOLOGY FORMED:")
    print("  Final connectivity: 0-1-2-3-4-5-0")
    print("  Ring path: 0→1→2→3→4→5→0")
    print("  Each qubit has exactly 2 neighbors")
    print()
    
    print("ADVANTAGES:")
    print("  ✓ Systematic layered approach")
    print("  ✓ Parallel gates in each layer → efficient")
    print("  ✓ Complete ring connectivity")
    print("  ✓ Even/odd symmetry → structured mixing")
    print("  ✓ Scalable to larger systems")

def visualize_connectivity():
    """Create a visual representation of the connectivity pattern."""
    
    print("\nCONNECTIVITY VISUALIZATION:")
    print("="*40)
    
    print("Qubit Layout:")
    print("  q0 ────── q1")
    print("   │         │")
    print("   │    q2 ─ q3")
    print("   │     │   │")
    print("   │     │   │")
    print("   └─ q5 ─── q4")
    print()
    
    print("Layer Breakdown:")
    print("  Even Layer: (0,1), (2,3), (4,5)")
    print("  Odd Layer:  (1,2), (3,4), (5,0)")
    print()
    
    print("Complete Ring Path:")
    print("  0 → 1 → 2 → 3 → 4 → 5 → 0")
    print("  ↑________________________|")

def main():
    """Generate and demonstrate the 6-qubit even/odd XY mixer."""
    
    print("6-QUBIT EVEN/ODD XY MIXER WITH CLOSED RING")
    print("="*55)
    print("Creating 6-qubit even/odd XY mixer...")
    print("Topology: Layered approach forming complete ring")
    print("Even Layer: (0,1), (2,3), (4,5)")
    print("Odd Layer:  (1,2), (3,4), (5,0) ← closes ring")
    print()
    
    # Plot the circuit
    qc_symbolic, qc_bound = plot_6qubit_even_odd_xy_mixer()
    
    # Demonstrate the effect
    demonstrate_6qubit_even_odd_effect()
    
    # Explain the pattern
    explain_even_odd_pattern()
    
    # Show connectivity
    visualize_connectivity()
    
    print("\n" + "="*55)
    print("6-QUBIT EVEN/ODD RING QAOA MIXER CONCEPT:")
    print("✓ Two layers: Even pairs + Odd connections")
    print("✓ Systematic approach: parallel then connecting")
    print("✓ Complete ring: every qubit connected in loop")
    print("✓ Efficient: parallelizable gates within layers")
    print("✓ Scalable: pattern extends to 8, 10, 12+ qubits")
    print("✓ Balanced: each qubit has exactly 2 neighbors")
    print("✓ QAOA ready: single β parameter controls all interactions")

if __name__ == "__main__":
    main()